# Import Library

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

---

In [2]:
root = os.path.join(os.getcwd(), 'DATA')

# Training Data
빅콘테스트 제공 데이터 전처리

## Import Data

In [3]:
df_raw = pd.read_excel(os.path.join(root, 'train.xlsx'))
df_train = df_raw.copy()

### Add Columns

In [4]:
# 날짜변수 추가
df_train['year'] = df_train['REG_DATE'].dt.year
df_train['month'] = df_train['REG_DATE'].dt.month
df_train['day'] = df_train['REG_DATE'].dt.day

In [5]:
df_train['VALUE_COUNT'] = 0;
value_dict = {}
for name, value in zip(df_train['P_NAME'].value_counts().index,df_train['P_NAME'].value_counts()):
    value_dict[name] = value

def value(col):
    return value_dict[col]

df_train['VALUE_COUNT'] = df_train['P_NAME'].apply(value)

### One hot encoding
P_IMPORT_TYPE 이라는 특수한 컬럼에 대한 전처리

In [6]:
import_type_list = set()
for tmp in df_train.P_IMPORT_TYPE.unique():
    for a in tmp.split(','):
        import_type_list.add(a)

In [7]:
for name in import_type_list:
    df_train[name] = 0
    df_train.loc[df_train['P_IMPORT_TYPE'].str.contains(name, regex=False), name] = 1

---

# Weather data

## Import Data

In [8]:
df_weather_code = pd.read_excel(os.path.join(root, 'raw_weather_code.xlsx'), header=None, index_col=0)

In [9]:
weather_list = [pd.read_csv(os.path.join(root, 'raw_weather_20151228_20161227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20161228_20171227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20171228_20181227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20181228_20191227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20191228_20201227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'raw_weather_20201228_20210818.csv'), encoding='euc-kr')]

## Preprocess
- '지점'에 따른 나라명 컬럼 (country) 추가
- 각 나라, 일자 별로 평균 강수량, 풍속, 기온 계산

In [10]:
# 지점에 따라 나라명 추가
def set_country(row):
    data = df_weather_code[df_weather_code[1] == row['지점']]
    if data.empty:
        return ""
    return data.iloc[0][2]


def preprocess_weather(df_weather):
    # 날짜 정보 정리
    df_weather['year'] = df_weather['일시'].astype('str').str[:4].astype('int')
    df_weather['month'] = df_weather['일시'].astype('str').str[5:7].astype('int')
    df_weather['day'] = df_weather['일시'].astype('str').str[8:10].astype('int')
    # 1차 평균
    df_weather['rain'] = df_weather[['지점', 'year', 'month', 'day', '강수량']].groupby(['지점', 'year', 'month', 'day']).transform('mean')
    df_weather['wind'] = df_weather[['지점', 'year', 'month', 'day', '풍속']].groupby(['지점', 'year', 'month', 'day']).transform('mean')
    df_weather['temperature'] = df_weather[['지점', 'year', 'month', 'day', '기온']].groupby(['지점', 'year', 'month', 'day']).transform('mean')
    # 컬럼/행 정리
    df_weather.drop(columns = ['지점명', '일시', '강수량', '풍속', '기온'], inplace=True)
    df_weather.drop_duplicates(inplace=True)
    # 나라명 추가
    df_weather['CTRY_1'] = ""
    for i, row in df_weather.iterrows():
        df_weather.at[i, 'CTRY_1'] = set_country(row)
    # 2차 평균
    df_weather['rain'] = df_weather[['CTRY_1', 'year', 'month', 'day', 'rain']].groupby(['CTRY_1', 'year', 'month', 'day']).transform('mean')
    df_weather['wind'] = df_weather[['CTRY_1', 'year', 'month', 'day', 'wind']].groupby(['CTRY_1', 'year', 'month', 'day']).transform('mean')
    df_weather['temperature'] = df_weather[['CTRY_1', 'year', 'month', 'day', 'temperature']].groupby(['CTRY_1', 'year', 'month', 'day']).transform('mean')
    # 컬럼/행 정리
    df_weather.drop(columns = ['지점'], inplace=True)
    df_weather.drop_duplicates(inplace=True)
    # 인덱스 정리
    df_weather.reset_index(drop=True, inplace=True)

In [11]:
for df in weather_list:
    preprocess_weather(df)
df_weather = pd.concat(weather_list)

In [12]:
df_weather = pd.concat(weather_list)

In [13]:
df_weather.describe()

,year,month,day,rain,wind,temperature
count,13947.000000,13947.000000,13947.000000,10303.000000,13947.000000,13947.000000
mean,2018.258407,6.276762,15.697426,-20.478450,3.312765,16.499339
std,1.613987,3.414373,8.809565,78.401760,5.434442,10.954919
min,2015.000000,1.000000,1.000000,-999.000000,-156.116667,-26.175000
25%,2017.000000,3.000000,8.000000,-6.701488,2.031250,9.912083
50%,2018.000000,6.000000,16.000000,1.035000,2.625000,19.356250
75%,2020.000000,9.000000,23.000000,3.676250,5.434846,25.244375
max,2021.000000,12.000000,31.000000,915.000000,15.125000,33.275000


#### 확인필요
- outlier 찾아내기 > 값이 너무 크거나 작은 경우 제외
- 날짜별로 확인 후 비어있는 값 채워넣기 (전/다음날 이용)
    - 13947개 데이터 중 rain, wind, temperature 갯수 보면 몇개 비어있는지 확인 가능
- 합치기..
- 강수량의 경우 NaN 값이 너무 많음
    - 위 전처리들을 한 후에도 많다면 사용불가
    - 위 전처리 후에는 적다면 전날/다음날 데이터 기반으로 채워넣기 진행

## Add to Training Data

In [14]:
df_train = pd.merge(df_train, df_weather, how='left', on=['year', 'month', 'day', 'CTRY_1'])

In [15]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,REG_DATE,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE,year,month,day,VALUE_COUNT,창난,줄기,외투막,필렛(F),지느러미,절단,한쪽껍질붙은,볼살,캐비아대용,꼬리_외화획득용,포장횟감,내장,횟감,머리,활,알,머리살,난포선,건조,껍질,개아지살,집게다리,간,다리,냉동,눈살,냉장,염장,살,슬라이스(S),동체,곤이,훈제,머리_외화획득용,자숙,목살,턱살,rain,wind,temperature
35262,2019-05-13,수산물,아르헨티나,중국,판매용,연체류 해물모듬,오징어,오징어,냉동,4.511682,2019,5,13,2150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
34351,2019-04-08,수산물,러시아,일본,판매용,알 곤이류,성게알,성게알,"냉장,횟감",241.875480,2019,4,8,453,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
35738,2019-05-27,수산물,시에라리온,시에라리온,판매용,어류,서대 박대 페루다,서대,냉동,3.984615,2019,5,27,359,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
32855,2019-02-18,수산물,세네갈,세네갈,판매용,연체류 해물모듬,갑오징어,갑오징어,냉동,4.600000,2019,2,18,783,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
31396,2018-12-24,수산물,중국,중국,판매용,어류,조기 보구치 강다리,참조기,냉동,9.478136,2018,12,24,206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.329386,2.255651,-0.703571


---

# Salinity

## Import Data

In [16]:
# df_salinity = pd.read_csv(os.path.join(root, 'raw_salinity.csv'))

## Drop Columns & Rows

In [17]:
# df_salinity.drop(df_salinity.columns[2], inplace=True, axis=1)
# df_salinity = df_salinity[(2015 <= df_salinity['obs_year']) & (df_salinity['obs_year'] <= 2021)]

2020, 2021 데이터의 부재로 인해 보류

---

# Oil

## Import Data

In [18]:
df_oil = pd.read_csv(os.path.join(root, 'raw_oil.csv'))
df_oil_dubai = pd.read_csv(os.path.join(root, 'raw_oil_dubai.csv'))
df_oil_brent = pd.read_csv(os.path.join(root, 'raw_oil_brent.csv'))

## Preprocess

In [19]:
def preprocess_oil(df):
    df['year'] = df['날짜'].str[:4].astype('int')
    df['month'] = df['날짜'].str[6:8].astype('int')
    df['day'] = df['날짜'].str[10:12].astype('int')
#     df['date'] = pd.to_datetime(df['날짜'].str[:4] + df['날짜'].str[6:8] + df['날짜'].str[10:12])
#     df = df.query('date.dt.dayofweek == 0')
    df.drop(columns = ['날짜', '오픈', '고가', '저가', '거래량', '변동 %'], inplace=True, axis=1)
    df.sort_index(ascending=False)
    return df

In [20]:
df_oil = preprocess_oil(df_oil)
df_oil_dubai = preprocess_oil(df_oil_dubai)
df_oil_brent = preprocess_oil(df_oil_brent)

## Check

In [21]:
print("oil:", len(df_oil), "\ndubai:", len(df_oil_dubai), "\nbrent:", len(df_oil_brent))

oil: 1451 
dubai: 1383 
brent: 1420


oil과 비교하였을 때 dubai의 경우 28개, brent의 경우 3개의 데이터가 적다  
따라서 df_oil 데이터를 사용하도록 한다

## Add to Training Data

In [22]:
df_train = pd.merge(df_train, df_oil, how='left', on=['year', 'month', 'day'])

In [23]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,REG_DATE,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE,year,month,day,VALUE_COUNT,창난,줄기,외투막,필렛(F),지느러미,절단,한쪽껍질붙은,볼살,캐비아대용,꼬리_외화획득용,포장횟감,내장,횟감,머리,활,알,머리살,난포선,건조,껍질,개아지살,집게다리,간,다리,냉동,눈살,냉장,염장,살,슬라이스(S),동체,곤이,훈제,머리_외화획득용,자숙,목살,턱살,rain,wind,temperature,종가
3800,2016-05-09,수산물,중국,중국,판매용,어류,메기 동자개,동자개,활,5.926949,2016,5,9,206,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.527439,2.363924,17.880967,43.44
29453,2018-10-22,수산물,인도네시아,인도네시아,판매용,젓갈류 해조류 해파리,해파리,해파리,염장,1.905328,2018,10,22,265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,69.17
28819,2018-10-01,수산물,불가리아,불가리아,판매용,패류 멍게류,고동,피뿔고둥,"냉동,살,자숙",8.559476,2018,10,1,246,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,NaN,NaN,NaN,75.30
19793,2017-11-27,수산물,미국,미국,판매용,어류,가오리,가오리,냉동,2.567153,2017,11,27,803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,58.11
30150,2018-11-12,수산물,태국,태국,판매용,연체류 해물모듬,갑오징어,갑오징어,냉장,9.744136,2018,11,12,783,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.377778,1.391667,28.003333,59.93


---

# Korea Weather

## Import Data

In [24]:
df_weather_kr = pd.read_csv(os.path.join(root, 'raw_weather_korea.csv'), encoding='euc-kr')

## Group
일자 별로 평균 구하기

In [25]:
df_weather_kr['wind_kr'] = df_weather_kr[['일시', '평균 풍속(m/s)']].groupby(['일시']).transform('mean')
df_weather_kr['temperature_kr'] = df_weather_kr[['일시', '평균 기온(°C)']].groupby(['일시']).transform('mean')
df_weather_kr['water_temp_kr'] = df_weather_kr[['일시', '평균 수온(°C)']].groupby(['일시']).transform('mean')

## Preprocess Date

In [26]:
df_weather_kr['year'] = df_weather_kr['일시'].str[:4].astype('int')
df_weather_kr['month'] = df_weather_kr['일시'].str[5:7].astype('int')
df_weather_kr['day'] = df_weather_kr['일시'].str[8:].astype('int')

## Drop Column
어차피 전부 해안가 대한민국이라 지점은 필요없다  
사용한 컬럼은 제거한다

In [27]:
drop = ['지점', '일시', '평균 풍속(m/s)', '평균 기온(°C)', '평균 수온(°C)']

In [28]:
df_weather_kr.drop(columns=drop, inplace=True, axis=1)

## Add to Training Data

In [29]:
df_train = pd.merge(df_train, df_weather_kr, how='left', on=['year', 'month', 'day'])

In [30]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,REG_DATE,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE,year,month,day,VALUE_COUNT,창난,줄기,외투막,필렛(F),지느러미,절단,한쪽껍질붙은,볼살,캐비아대용,꼬리_외화획득용,포장횟감,내장,횟감,머리,활,알,머리살,난포선,건조,껍질,개아지살,집게다리,간,다리,냉동,눈살,냉장,염장,살,슬라이스(S),동체,곤이,훈제,머리_외화획득용,자숙,목살,턱살,rain,wind,temperature,종가,wind_kr,temperature_kr,water_temp_kr
88122,2016-07-04,수산물,중국,중국,판매용,어류,복어,흑밀복,냉동,0.918242,2016,7,4,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,4.915931,2.625000,24.874548,48.65,5.035294,21.864706,20.788235
659255,2019-10-07,수산물,베트남,베트남,판매용,연체류 해물모듬,갑오징어,갑오징어,냉동,4.020221,2019,10,7,783,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-85.827083,2.250000,29.042188,52.75,5.250000,19.792857,22.525000
339191,2017-12-18,수산물,중국,중국,판매용,어류,복어,참복,냉동,3.521023,2017,12,18,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,-9.881667,2.358725,-1.692473,57.16,6.994118,6.105882,13.017647
553961,2019-03-04,수산물,인도네시아,인도네시아,판매용,연체류 해물모듬,문어,문어,냉동,6.473073,2019,3,4,486,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,56.59,5.517647,9.523529,10.911765
630340,2019-08-05,수산물,이탈리아,일본,판매용,어류,참치 새치류,참다랑어,"냉동,필렛(F),횟감",33.155360,2019,8,5,979,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,54.69,3.252941,26.406250,26.218750


---

# 소비자물가지수

## Import Data

In [31]:
df_cpi_tmp1 = pd.read_excel(os.path.join(root, 'raw_cpi_region.xlsx'), encoding='euc-kr')  # 시도별 CPI
df_cpi_tmp2 = pd.read_excel(os.path.join(root, 'raw_cpi_purpose.xlsx'), encoding='euc-kr')  # 품목별 CPI

## Drop Columns/Rows

In [32]:
df_cpi_tmp1 = df_cpi_tmp1[df_cpi_tmp1['시도별(1)'] == '전국'].drop(columns=['시도별(1)', '시도별(2)','품목별(1)'], axis=1)

In [33]:
df_cpi_tmp2 = df_cpi_tmp2.iloc[[2,4],:].drop(columns=['시도별'],axis=1)  # 식료품 및 음식서비스 컬럼 추출 &시도별 칼럼 제거

## Pivoting

In [34]:
df_cpi1 = pd.DataFrame(df_cpi_tmp1.values.reshape(49,1),index=df_cpi_tmp1.columns, columns=['CPI'])

In [35]:
df_cpi2 = pd.DataFrame(df_cpi_tmp2.iloc[0,:].values,index = df_cpi_tmp2.columns,columns=['CPI_food'])
df_cpi2['CPI_foodservice'] = df_cpi_tmp2.iloc[1,:].values

## Preprocess Date

In [36]:
df_cpi2.drop('지출목적별',axis=0,inplace=True) # 지출목적별 행 삭제

In [37]:
df_cpi1.reset_index(inplace=True)  # 날짜 인덱스 > 컬럼으로 변경
df_cpi2.reset_index(inplace=True)  # 날짜 인덱스 > 컬럼으로 변경

In [38]:
df_cpi1['year'] = df_cpi1['index'].str[:4].astype('int')
df_cpi1['month'] = df_cpi1['index'].str[5:].astype('int')
df_cpi1.drop(columns=['index'], inplace=True, axis=1)

df_cpi2['year'] = df_cpi2['index'].str[:4].astype('int')
df_cpi2['month'] = df_cpi2['index'].str[5:].astype('int')
df_cpi2.drop(columns=['index'], inplace=True, axis=1)

## Add to Training Data

In [39]:
df_train = pd.merge(df_train, df_cpi1, how='left', on=['year', 'month'])
df_train = pd.merge(df_train, df_cpi2, how='left', on=['year', 'month'])

In [40]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,REG_DATE,P_TYPE,CTRY_1,CTRY_2,P_PURPOSE,CATEGORY_1,CATEGORY_2,P_NAME,P_IMPORT_TYPE,P_PRICE,year,month,day,VALUE_COUNT,창난,줄기,외투막,필렛(F),지느러미,절단,한쪽껍질붙은,볼살,캐비아대용,꼬리_외화획득용,포장횟감,내장,횟감,머리,활,알,머리살,난포선,건조,껍질,개아지살,집게다리,간,다리,냉동,눈살,냉장,염장,살,슬라이스(S),동체,곤이,훈제,머리_외화획득용,자숙,목살,턱살,rain,wind,temperature,종가,wind_kr,temperature_kr,water_temp_kr,CPI,CPI_food,CPI_foodservice
701223,2019-12-23,수산물,러시아,러시아,판매용,갑각류,게,대게,냉장,6.248869,2019,12,23,603,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,60.52,7.052381,9.338095,13.947619,105.12,109.56,110.52
541064,2019-01-28,수산물,아르헨티나,아르헨티나,판매용,어류,홍어,홍어,냉동,3.703274,2019,1,28,746,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,51.99,8.952941,6.525000,11.262500,104.24,109.22,109.86
193647,2017-02-20,수산물,뉴질랜드,뉴질랜드,판매용,알 곤이류,호키(새꼬리민태)알,새꼬리민태알,"냉동,알",2.878231,2017,2,20,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,53.97,11.593750,6.112500,10.173333,102.92,107.73,104.11
234541,2017-05-15,수산물,러시아,러시아,판매용,패류 멍게류,조개 백합 대합,북방대합,활,2.650000,2017,5,15,134,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,48.85,5.075000,15.850000,15.362500,102.83,105.34,104.88
65040,2016-05-16,수산물,베트남,베트남,판매용,연체류 해물모듬,오징어,화살오징어,건조,25.773991,2016,5,16,512,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.8,2.1875,27.46875,47.72,6.647059,14.923529,15.100000,100.81,101.39,102.43


---

# Final

## One-hot Encoding

In [41]:
one_hot = ['CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1', 'CATEGORY_2', 'P_NAME']

In [42]:
df_train = pd.get_dummies(df_train, columns=one_hot)

## Drop Columns

In [43]:
drop = ['REG_DATE', 'P_TYPE', 'P_IMPORT_TYPE']

In [44]:
df_train.drop(columns = drop, inplace=True)

In [45]:
pd.set_option("display.max_columns", None)
df_train.sample(5)

,P_PRICE,year,month,day,VALUE_COUNT,창난,줄기,외투막,필렛(F),지느러미,절단,한쪽껍질붙은,볼살,캐비아대용,꼬리_외화획득용,포장횟감,내장,횟감,머리,활,알,머리살,난포선,건조,껍질,개아지살,집게다리,간,다리,냉동,눈살,냉장,염장,살,슬라이스(S),동체,곤이,훈제,머리_외화획득용,자숙,목살,턱살,rain,wind,temperature,종가,wind_kr,temperature_kr,water_temp_kr,CPI,CPI_food,CPI_foodservice,CTRY_1_가나,CTRY_1_감비아,CTRY_1_그리스,CTRY_1_그린란드,CTRY_1_기니,CTRY_1_기니비사우,CTRY_1_나미비아,CTRY_1_남아프리카 공화국,CTRY_1_네덜란드,CTRY_1_노르웨이,CTRY_1_뉴질랜드,CTRY_1_니카라과,CTRY_1_대만,CTRY_1_대한민국,CTRY_1_라이베리아,CTRY_1_라트비아,CTRY_1_러시아,CTRY_1_루마니아,CTRY_1_리비아,CTRY_1_마다가스카르,CTRY_1_말레이시아,CTRY_1_멕시코,CTRY_1_모로코,CTRY_1_모리타니,CTRY_1_모잠비크,CTRY_1_몰타,CTRY_1_미국,CTRY_1_미얀마,CTRY_1_미크로네시아 연방,CTRY_1_바누아투,CTRY_1_바레인,CTRY_1_방글라데시,CTRY_1_베네수엘라,CTRY_1_베트남,CTRY_1_불가리아,CTRY_1_브라질,CTRY_1_사우디아라비아,CTRY_1_사이프러스,CTRY_1_세네갈,CTRY_1_세이셸,CTRY_1_세인트빈센트 그레나딘,CTRY_1_소말리아,CTRY_1_수리남,CTRY_1_스리랑카,CTRY_1_스페인,CTRY_1_시에라리온,CTRY_1_싱가포르,CTRY_1_아랍에미리트,CTRY_1_아르헨티나,CTRY_1_아이슬란드,CTRY_1_아일랜드,CTRY_1_알제리,CTRY_1_앙골라,CTRY_1_에스토니아,CTRY_1_에콰도르,CTRY_1_영국,CTRY_1_오만,CTRY_1_우루과이,CTRY_1_우크라이나,CTRY_1_이란,CTRY_1_이집트,CTRY_1_이탈리아,CTRY_1_인도,CTRY_1_인도네시아,CTRY_1_일본,CTRY_1_중국,CTRY_1_칠레,CTRY_1_캐나다,CTRY_1_콜롬비아,CTRY_1_쿠바,CTRY_1_쿡 제도,CTRY_1_크로아티아,CTRY_1_키리바시,CTRY_1_태국,CTRY_1_터키,CTRY_1_튀니지,CTRY_1_파나마,CTRY_1_파키스탄,CTRY_1_파푸아뉴기니,CTRY_1_팔라우,CTRY_1_페루,CTRY_1_포르투갈,CTRY_1_포클랜드 제도,CTRY_1_프랑스,CTRY_1_피지,CTRY_1_필리핀,CTRY_1_호주,CTRY_2_가나,CTRY_2_감비아,CTRY_2_그리스,CTRY_2_기니,CTRY_2_기니비사우,CTRY_2_기타(ZZ),CTRY_2_나미비아,CTRY_2_남아프리카 공화국,CTRY_2_네덜란드,CTRY_2_노르웨이,CTRY_2_뉴질랜드,CTRY_2_대만,CTRY_2_덴마크,CTRY_2_독일,CTRY_2_라이베리아,CTRY_2_러시아,CTRY_2_루마니아,CTRY_2_마다가스카르,CTRY_2_말레이시아,CTRY_2_멕시코,CTRY_2_모로코,CTRY_2_모리셔스,CTRY_2_모리타니,CTRY_2_모잠비크,CTRY_2_몰타,CTRY_2_미국,CTRY_2_미얀마,CTRY_2_미크로네시아 연방,CTRY_2_바누아투,CTRY_2_바레인,CTRY_2_방글라데시,CTRY_2_베네수엘라,CTRY_2_베트남,CTRY_2_벨리즈,CTRY_2_북한,CTRY_2_불가리아,CTRY_2_브라질,CTRY_2_사모아,CTRY_2_사우디아라비아,CTRY_2_세네갈,CTRY_2_세이셸,CTRY_2_솔로몬 제도,CTRY_2_수리남,CTRY_2_스리랑카,CTRY_2_스웨덴,CTRY_2_스페인,CTRY_2_시에라리온,CTRY_2_싱가포르,CTRY_2_아랍에미리트,CTRY_2_아르헨티나,CTRY_2_아이슬란드,CTRY_2_아일랜드,CTRY_2_앙골라,CTRY_2_에콰도르,CTRY_2_영국,CTRY_2_오만,CTRY_2_우루과이,CTRY_2_우크라이나,CTRY_2_이란,CTRY_2_이집트,CTRY_2_이탈리아,CTRY_2_인도,CTRY_2_인도네시아,CTRY_2_일본,CTRY_2_중국,CTRY_2_지부티,CTRY_2_칠레,CTRY_2_캐나다,CTRY_2_콜롬비아,CTRY_2_쿠바,CTRY_2_크로아티아,CTRY_2_키리바시,CTRY_2_태국,CTRY_2_터키,CTRY_2_투발루,CTRY_2_튀니지,CTRY_2_파나마,CTRY_2_파키스탄,CTRY_2_파푸아뉴기니,CTRY_2_팔라우,CTRY_2_페루,CTRY_2_포르투갈,CTRY_2_포클랜드 제도,CTRY_2_프랑스,CTRY_2_피지,CTRY_2_필리핀,CTRY_2_호주,CTRY_2_홍콩,P_PURPOSE_반송품(기타),P_PURPOSE_외화획득용 원료,P_PURPOSE_자사제품제조용,P_PURPOSE_판매용,P_PURPOSE_합작,CATEGORY_1_갑각류,CATEGORY_1_기타 수입식품,CATEGORY_1_알 곤이류,CATEGORY_1_어류,CATEGORY_1_연체류 해물모듬,CATEGORY_1_젓갈류 해조류 해파리,CATEGORY_1_패류 멍게류,CATEGORY_2_가리비,CATEGORY_2_가물치,CATEGORY_2_가사리,CATEGORY_2_가오리,CATEGORY_2_가자미,CATEGORY_2_가재 랍스타,CATEGORY_2_갈치,CATEGORY_2_갑오징어,CATEGORY_2_개복치,CATEGORY_2_개불,CATEGORY_2_게,CATEGORY_2_고동,CATEGORY_2_고등어,CATEGORY_2_고시래기,CATEGORY_2_골뱅이,CATEGORY_2_광어 넙치,CATEGORY_2_김,CATEGORY_2_꼬막,CATEGORY_2_꼴뚜기,CATEGORY_2_꽁치 학꽁치,CATEGORY_2_꽁치 확꽁치,CATEGORY_2_낙지,CATEGORY_2_날치알,CATEGORY_2_남극빙어,CATEGORY_2_노래미,CATEGORY_2_농어,CATEGORY_2_능성어 붉바리 바리,CATEGORY_2_다시마,CATEGORY_2_달고기,CATEGORY_2_대구,CATEGORY_2_대구알,CATEGORY_2_도다리,CATEGORY_2_도미 감성돔 돔류,CATEGORY_2_망둑어,CATEGORY_2_멍게,CATEGORY_2_메기 동자개,CATEGORY_2_메로,CATEGORY_2_멸치,CATEGORY_2_명란(명태알),CATEGORY_2_명태,CATEGORY_2_문어,CATEGORY_2_물메기(곰치),CATEGORY_2_미꾸라지,CATEGORY_2_미역,CATEGORY_2_민물붕어,CATEGORY_2_민어 점성어,CATEGORY_2_밀크피시,CATEGORY_2_바지락,CATEGORY_2_방어,CATEGORY_2_밴댕이,CATEGORY_2_버터플라이 킹피쉬,CATEGORY_2_벤자리 알롱이,CATEGORY_2_병어,CATEGORY_2_보리멸,CATEGORY_2_복어,CATEGORY_2_부세,CATEGORY_2_붉평치(만다이 꽃돔),CATEGORY_2_삼치,CATEGORY_2_상어 고래,CATEGORY_2_새우,CATEGORY_2_샛돔류알,CATEGORY_2_서대 박대 페루다,CATEGORY_2_성게알,CATEGORY_2_소라,CATEGORY_2_송어,CATEGORY_2_쏘가리,CATEGORY_2_아귀,CATEGORY_2_양미리 정어리,CATEGORY_2_양태,CATEGORY_2_어류 기타,CATEGORY_2_연어,CATEGORY_2_연어알,CATEGORY_2_열빙어(시샤모),CATEGORY_2_열빙어(시샤모)알,CATEGORY_2_오징어,CATEGORY_2_옥돔,CATEGORY_2_우럭 볼락,CATEGORY_2_우렁 다슬기,CATEGORY_2_은민대구알,CATEGORY_2_임연수,CATEGORY_2_잉어,CATEGORY_2_자라,CATEGORY_2_장어,CATEGORY_2_재첩,CATEGORY_2_적어 눈볼대,CATEGORY_2_전갱기 매가리,CATEGORY_2_전갱이 매가리,CATEGORY_2_전복,CATEGORY_2_전어,CATEGORY_2_조개,CATEGORY_2_조개 백합 대합,CATEGORY_2_조기 보구치 강다리,CATEGOR

## Save Files

In [46]:
def save_file(df, file_name):
    df.to_csv(os.path.join(root, file_name), encoding='utf-8')

In [48]:
save_file(df_weather, 'preprocessed_weather.csv')  # 제조국 날씨
save_file(df_oil, 'preprocessed_oil.csv')  # 원유 종가
save_file(df_weather_kr, 'preprocessed_weather_korea.csv')  # 한국 날씨
save_file(df_cpi1, 'preprocessed_cpi_region.csv')  # 전체 소비자물가지수
save_file(df_cpi2, 'preprocessed_cpi_purpose.csv')  # 음식/음식서비스 소비자물가지수
save_file(df_train, 'preprocessed_train.csv')  # 최종 df